In [10]:
import requests #for downloading the web page

In [9]:
import pandas as pd #манипуляции с таблицами

In [2]:
import time
import os #for file management and sleep while scraping

In [7]:
import numpy as np #как генератор чисел

In [8]:
from bs4 import BeautifulSoup #for getting text data from html. Главный рабочий инструмент.

In [ ]:
# Далее я показываю несколько примеров. Сам я много практиковался в получении нужных значений, а потом перевёл в функции.

In [15]:
urlPsyBasic = 'https://www.b17.ru/andrey_klementyev/'

In [20]:
page = requests.get(urlPsyBasic)

In [21]:
page.status_code #ахаха, он был заблочен на дубковском интернете (мне кажется, из-за меня), был код 503.

200

In [22]:
soup = BeautifulSoup(page.content, 'lxml')

In [23]:
namePsych = soup.title.string #вот так, к примеру, можно выгрузить имя и проблемы. Решается частично умом, частично перебором.

In [24]:
problemsPsych = soup.p.text

In [66]:
max(soup.find_all('div', class_='response_list')[62].text.splitlines()) #можно так, но это непрактично.

'Алёна Викторовна, даже не думайте, мне очень дорого Ваше мнение, и так, как и я раскрываюсь Вам правдиво и искренне, точно так же я ценю и Ваше искреннее и правдивое мнение, мысли, вопросы и советы. Без этого наше общение было бы абсолютно бесполезным. Прошло совсем немного времени, как мы с Вами общаемся, а я каждый день чувствую непреодолимое желание заглянуть на сайт в надежде на новые Ваши сообщения)Отзыв из текста демо-консультации на форумеБыла 1 демо-консультация с пользователем.'

In [65]:
all_text = soup.find_all('div', {"class" : "response_list"})[3].find('div', class_='t').text #выгружает корректно!
#и сохраняется количество комментариев. Но смешивается ответ психолога с отзывом.

In [67]:
all_text.split('Ответ специалиста:')[1] #разделяем. Отлично, работает.

'Спасибо за ваш отклик. Да, первичная консультация по телефону бесплатная. Это помощь и ориентации в проблематике.К сожалению, я не консультирую бесплатно, кроме дело-консультации на форуме.'

In [277]:
# я очень постепенно убирал всё лишнее из строки. И только так смог выгрузить их. Наверное, можно было сделать и покороче. Но работает беспрекословно, ошибок не выдаёт.
def city_find(a = soup.find_all('div', class_='status-grey')):
    cities = []
    for i in a:
        cities.append(i.text)
    cities_clean = []

    for i in cities:
        for j in i.split('г. ')[1:]:
            cities_clean.append([x.strip() for x in j.split(',')])

    cities_row = []
    for i in cities_clean:
        for j in i:
            if j == '':
                del j
            else:
                cities_row.append(j)
    return [', '.join(cities_row)]

In [ ]:
# получает кусок текста всех отзывов + ответы одной страницы. Делит на отзыв и ответ специалиста. И возвращает два листа с отзывами и комментариями.
# отзывы и ответы одной длины. При отсутствии ответа в список добавляется пустой элемент.
def all_text_clean(all_text):
    all_comments = []
    all_answers = []
    for i in all_text: 
        if 'Ответ специалиста:' in i:
            all_comments.append(i.split('Ответ специалиста:')[0])
            all_answers.append(i.split('Ответ специалиста:')[1])
        else:
            all_comments.append(i)
            all_answers.append('')
    return all_comments, all_answers

In [ ]:
# убрал упоминания о демо-консультациях. Они вставляются сайтом автоматически. Получаю все отзывы как лист, возвращаю тоже их.
def cleaning_list_comments(all_comments):
    for i in range(len(all_comments)):
        if 'Была 1 демо-консультация' in all_comments[i]:
            all_comments[i] = all_comments[i].split('Была 1 демо-консультация')[0]
    for i in range(len(all_comments)):
        if 'Было 2 демо-консультации с пользователем' in all_comments[i]:
            all_comments[i] = all_comments[i].split('Было 2 демо-консультации с пользователем')[0]
    for i in range(len(all_comments)):
        if 'Отзыв из текста демо-консультации' in all_comments[i]:
            all_comments[i] = all_comments[i].split('Отзыв из текста демо-консультации')[0]
    return all_comments

In [ ]:
#далее вы увидите много переменных, листов. Их я тестировал вне функции, а потом занёс внутрь.

In [755]:
# в эту функцию загружаем id вида /something/ . Он выгружает dataframe с отзывами и ещё некоторыми данными этого психолога. Только папочку лучше заранее создать.
def table_from_id_psycho(ids):
    url = str('https://www.b17.ru' + ids) #link on the web page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml') #выгрузил данные
    all_text = []
    for i in range(len(soup.find_all('div', {"class" : "response_list"}))):
        all_text.append(soup.find_all('div', {"class" : "response_list"})[i].find('div', class_='t').text)
 #разделил комментарии   
    a = 2 + np.random.randn()
    if a < 0:
        time.sleep(2)
    else:
        time.sleep(a)
# необходима пауза, чтобы сайт не заблочил. Я такую ошибку сделал лишь однажды. Теперь на дубковском интернете b17 заблочен forever
    all_comments, all_answers = all_text_clean(all_text)
# разделил отзывы
# далее я выгружаю ещё некоторые данные. Так как там может их не быть, выпадает ошибка. Поэтому использую такую конструкцию.
    try:
        numberPubl = int(soup.find_all(id ='vkl_article')[0].find(class_='n').text)
    except:
        numberPubl = 'error'
    try:
        numberOtz = int(soup.find_all(id ='vkl_otzyv')[0].find(class_='n').text) 
    except:
        numberOtz = 'error'
    try:
        numberObratilis = int(soup.find_all(class_='box-admin')[0].text.split(' ')[1])
    except:
        numberObratilis = 'error'
    try:
        numberOnlKons = int(soup.find_all(class_='box-admin')[1].text.split(' ')[1])
    except:        
        numberOnlKons = 'error'
# город и длина всех отзывов, чтобы растянуть значения такие, как город и цифры.
    city = city_find(soup.find_all('div', class_='status-grey'))
    lenl = len(all_comments)
    
    return pd.DataFrame({'Comments': cleaning_list_comments(all_comments), 'Answers': all_answers, 'Link': [url]*lenl, 'Cities': city*lenl, 'Number of publications':[numberPubl]*lenl, 'Number of comments':[numberOtz]*lenl, 'Обратились': [numberObratilis]*lenl, 'Number of consultations': [numberOnlKons]*lenl})
# .to_excel(r'C:\Users\Professional\Jupyter Notebooks\parsPsy_left_comrades\{}.xlsx'.format(ids[1:len(ids)-1]), index = False) Лучше выгружать в файл вне функции, так как мультифункциональность внутри одной функции не есть хорошо.
# , 'comrad': [list_of_equal_work_change.pop()]*lenl Я распределял сначала работу прямо здесь, но это непрактично.

In [388]:
# я уже забываю для чего мне нужны были функции ниже

In [ ]:
# у нас есть словарь с id на каждой странице. Мы загружаем страницу, а он нам все id на этой странице.
def ids_on_page(page):
    return ids_dict[str(page)]

In [5]:
try:
    os.mkdir('testtest11')
except:
    pass
#код, чтобы создать папку и не получить ошибку в то же время.

In [ ]:
# выгрузка всех id со страницы. Научимся с 1, сделаем для всех остальных. Они однотипные.

In [12]:
urlMain = 'https://www.b17.ru/otzyv/online/?page=66' 

In [13]:
pageMain = requests.get(urlMain)

In [14]:
soupMain = BeautifulSoup(pageMain.content, 'lxml')

In [15]:
endPage = int(soupMain.find(class_='page-list').text.split(' ')[-1]) #так мы можем выгрузить последнюю страницу

In [18]:
ids = soupMain.find_all(class_='f') # попытка хорошая, но надо дальше выгружать данные с аттрибутом href

In [16]:
pages = np.arange(1, endPage + 1).tolist()

In [731]:
basic_url = 'https://www.b17.ru/otzyv/online/?page='

In [732]:
all_url_pages = [basic_url + str(i) for i in pages] # список ссылкок на все страницы. От 1 до последней.

In [733]:
# на вход: список полных url на страницы вида 'https://www.b17.ru/otzyv/online/?page=66'
# выход: словарь вида {'страница':[список id психологов на этой странице]}
def all_url_psychos_with_page(list_of_urls):
    ids_dict = {}
    for i in list_of_urls:
        page = requests.get(i)
        soup = BeautifulSoup(page.content, 'lxml')
        fastlist = soup.find_all('div', class_='f') # при помощи fastlist я выгружаю собственно id психологов
        page_num = i.split('=')[1]
        ids_list = []
        for i in fastlist:
            ids_list.append(i.a.get('href'))
        a = 2 + np.random.randn() # рандомный сон
        if a < 0:
            time.sleep(2)
        else:
            time.sleep(a)
#         print(len(ids_dict), page_num, ids_list[0]) Это проверка работоспособности в реальном времени.
        ids_dict[page_num] = ids_list
    return ids_dict

In [20]:
ids_dict = all_url_psychos_with_page(all_url_pages)

In [865]:
with open("ids_dict_09_03.txt", "w") as f:
    f.write(str(ids_dict))
#так мы можем сохранить словарь локально, потому что список выгружается около 15 минут.

In [6]:
dict08 = {}
with open("ids_dict_08_03.txt", "r") as f:
    dict08 = eval(f.read())
# а так выгружаем словарь из файла.

In [866]:
# all_id_unique = pd.Series(all_id).unique()

In [738]:
# получает словарь как из all_url_psychos_with_page(), и возвращает список всех id
def all_id(some_dict):
    all_id = []
    for i in range(1, len(ids_dict) + 1):
        for j in ids_dict[str(i)]:
            all_id.append(j)
    return all_id

In [739]:
all_id_unique = pd.Series(all_id(ids_dict)).unique() #так находим уникальные id

In [432]:
pd.Series(all_id(ids_dict)).value_counts() # ага, повторы есть

/cvetkova_ni/        149
/a_vyukov/           108
/kuzmichev/           98
/a2n/                 96
/volira/              94
                    ... 
/merilyn1981/          3
/volgina/              2
/sergeeva_milena/      1
/qweasd/               1
/chvalun/              1
Length: 297, dtype: int64

In [ ]:
len(all_id_unique) # что выдаст 297. Всего 297 психологов, а список id подряд длины около 7300.

In [294]:
with open("ids_unique.txt", "w") as f:
    for s in all_id_unique:
        f.write(str(s) +"\n")
# так я ещё и сохранил уникальные id в файл.

In [322]:
list_unique_ids = []
with open("ids_unique.txt", "r") as f:
    for line in f:
        list_unique_ids.append(line.strip())

In [ ]:
# так как рационально запарсить? Всего 297 id, мы по ним проходимся и выгружаем 297 таблицы excel.

In [ ]:
for i in all_id_unique:
    table_from_id_psycho(i).to_excel(r'C:\Users\Professional\Jupyter Notebooks\parsPsyAll297\{}.xlsx'.format(i[1:len(i)-1]), index = False)

In [ ]:
#спустя минут 15 выгрузятся все 297 психолога. Отлично!

In [ ]:
# далее я разбираюсь с тем, какую кому страницу кодировать. Ведь парсер не всё умеет копировать. Для остального нужны люди.

In [22]:
comrads = pd.read_excel('comrads.xlsx') # Да, правильно писать comrades. Это моя ошибка.
comrads = comrads.iloc[:, 1:3]

In [24]:
comrads.head(4)

,Учащийся,страницы выдачи отзывов. От следующей страницы предыдущего участника до указанной (только кодируют)
0,Аблямитова Милена,2
1,Ананьева Евдокия,4
2,Белоусова Алена,6
3,Болдыгин Даниил,8


In [25]:
columns_double = [x for x in list(comrads.iloc[:,1]) if type(x) == int]  # создаю дубль колонки со страницами.

In [26]:
columns_double.insert(0, 0) #и её сдвигаю.

In [27]:
comrads['first'] = pd.Series(map(lambda x: 1 + x, columns_double)) # теперь они стоят рядом, правда тип в first float, но эти значения временные всё равно.

In [28]:
comrads.head(5)

,Учащийся,страницы выдачи отзывов. От следующей страницы предыдущего участника до указанной (только кодируют),first
0,Аблямитова Милена,2,1.0
1,Ананьева Евдокия,4,3.0
2,Белоусова Алена,6,5.0
3,Болдыгин Даниил,8,7.0
4,Веселовский Леонид,10,9.0


In [29]:
# Функция получает весь dataframe, потом возвращает лист с листами из страниц каждого учащегося
def list_of_pages(comrads): 
    comrads_temp = comrads.iloc[:174, 1:]
    b = []
    for i in range(174):
        first = int(comrads_temp.iloc[i, 1])
        end = int(comrads_temp.iloc[i, 0])
        k = []
        for i in range(first, end + 1):
            k.append(i)
        b.append(k)
    return b

In [30]:
all_list_of_pages = list_of_pages(comrads)

In [31]:
comrads = comrads.rename(columns = {'страницы выдачи отзывов. От следующей страницы предыдущего участника до указанной (только кодируют)' : 'end', 'double_pages':'first'})

In [32]:
comrads['pages'] = pd.Series(all_list_of_pages)

In [33]:
comrads = comrads.iloc[:174, :]

In [34]:
comrads[comrads.Учащийся.str.contains('Ерм')] # и выписываю 32 в следующие листы

,Учащийся,end,first,pages
32,Ермаков Павел,66,65.0,"[65, 66]"


In [35]:
comrads_id_list = [32] # и так далее, здесь хранятся id учащихся. Это кто с самого начала присутствовал. Их нет, потому что я не хочу разглашать имена.

In [19]:
left_comrades = [] # им я выделил оставшихся психологов

In [32]:
add_comrades = [] # кто присоединился к основной таблице, но позже

21

In [36]:
comrads.iloc[comrads_id_list,:].drop(['end','first'],axis = 1).sort_values(by = 'pages')

,Учащийся,pages
32,Ермаков Павел,"[65, 66]"


In [24]:
# получает таблицу сomrads и список учащихся. Выгружает список всех страниц.
def all_pages(comrads, list_of_comrads = comrads_id_list):
    all_pages = []
    for i in comrads.iloc[list_of_comrads,:]['pages']:
        for j in i:
            all_pages.append(j)
    return all_pages

In [25]:
# получает список всех страниц, выгружает список всех id психологов
def all_ids_by_pages(all_pages):
    all_test_ids = []
    for i in all_pages(comrads):
        all_test_ids.append(ids_on_page(i)) # здесь красивее было бы использовать extend, а не append
    return [x for y in all_test_ids for x in y]

In [ ]:
# получает список всех id, возвращает только уникальные. set() тоже должен неплохо работать, но ладно.
def all_ids_by_pages_unique(ids_all):
    return pd.Series(ids_all).unique()

In [742]:
len(all_ids_by_pages_unique(all_ids_by_pages(all_pages))) # я вообще кричал. Думал, что можно сделать это дело рационально.
# но не так много людей покрыли почти всех психологов (87.9%)

261

In [476]:
# получает полную таблицу учащихся. Возвращает список с именами людей, растянутыми на всех психологов (261).
def list_of_equal_work(comrads, comrads_list = comrads_id_list):
    list_of_working_comrads = [x for x in comrads.iloc[comrads_list,0]]
    list_of_working_comrads.remove('Ермаков Павел')  # у него других дел немало))
    list_all_work = []
    min_work = len(all_ids_by_pages_unique(all_ids_by_pages(all_pages))) // len(list_of_working_comrads) + 1
    left_work = len(all_ids_by_pages_unique(all_ids_by_pages(all_pages))) - (len(list_of_working_comrads) - 1) * min_work
    for i in list_of_working_comrads:
        if i == 'Здесь имя одного человека': # он получит остаток психологов
            list_all_work.extend([i]*left_work)
        else:
            list_all_work.extend([i]*min_work)
            # я заменил append на extend для лаконичности, должно работать.
#     for i in list_of_working_comrads:
#         if i == '':
#             list_all_work.append([i]*left_work)
#         else:
#             list_all_work.append([i]*min_work)
#     return [x for y in list_all_work for x in y]
    return list_all_work

In [518]:
list_of_equal_work_change = list_of_equal_work(comrads)

In [529]:
# я работу распределял при парсинге, но мне кажется, что это неправильно, я удалил, поэтому кода нет. Только в комментах под той большой функцией. Лучше создать полную таблицу.
# в ней сгруппировать по id психологов и так же рассчитать количество работы, не забыть убрать сортировку, которая стоит по умолчанию.

In [ ]:
# код ниже отвечает за то, чтобы собрать все файлы вместе в один. А потом из одного сделать много отдельных файлов.

In [24]:
# даю список id, он возвращает список dataframes, выгруженных локально.
def list_of_tables(all_ids):
    table = []
    for i in all_ids:
        i = i.strip('/')
        table.append(pd.read_excel(r'C:\Users\Professional\Jupyter Notebooks\parsPsy7\{}.xlsx'.format(i))) # прописываем папку с отдельными файлами
    return table

In [525]:
table_of_comrads = list_of_tables(all_ids_by_pages_unique(all_ids_by_pages(all_pages)))
# это лист таблиц с уникальными психологами, которые и необходимы для минимализации работы.

In [25]:
# эта функция берёт лист таблиц, объединяет и сохраняет локально.
def table_list_to_xlsx(table, name):
    os.mkdir(str(name))
    pd.concat(table, axis = 0).to_excel(r'C:\Users\Professional\Jupyter Notebooks\{}\{}.xlsx'.format(name, name), index = False)

In [526]:
table_list_to_xlsx(table_of_comrads, 'final') # на выходе один файл, около 9300 строк.

In [34]:
# возвращает словарь с именем студента и листом его психологов.
# но перед этим надо запарсить словарь с ссылками на психологов на всех страницах.
def dict_of_ids_of_each_comrad(comrads, what_dict, comrads_list = comrads_id_list):
    df_basic = comrads.iloc[comrads_list,:].drop(['end','first'], axis = 1).sort_values(by = 'pages')
    dict_of_ids = {}
    for i in range(len(df_basic)):
        dict_of_ids[df_basic.iloc[i, 0]] = ids_on_list_of_pages(df_basic.iloc[i, 1], what_dict)
    return dict_of_ids

In [35]:
# получает лист страниц. А возвращает лист всех id на этих страницах.
def ids_on_list_of_pages(list_of_pages, what_dict):
    fastlist = []
    for i in list_of_pages:
        fastlist.append(what_dict[str(i)]) # вроде можно упростить при помощи extend, но здесь уже не рискую.
    return [x for y in fastlist for x in y]

In [602]:
# for key, value in dict1.items():
#     print(key, value)

In [52]:
# получает список dataframe, объединяет и сохраняет локально.
def table_list_to_xlsx(table, name, name_root = 'final'):
    pd.concat(table, axis = 0).to_excel(r'C:\Users\Professional\Jupyter Notebooks\{}\{}.xlsx'.format(name_root, name), index = False)

In [55]:
# финишная функция! Выгружает по отдельным файлам нужных психологов с нужным именем файла.
def all_tables_final(table, what_dict, comrads_list = comrads_id_list, name_root = 'all_final'):
    try:
        os.mkdir(name_root)
    except:
        pass
    dict_all = dict_of_ids_of_each_comrad(comrads, what_dict)
    for key, values in dict_all.items():
        list_of_tables_final_one = []
        for i in values:
            list_of_tables_final_one.append(table[table.iloc[:,3].str.contains(i)])
        table_list_to_xlsx(list_of_tables_final_one, name = key, name_root)

In [56]:
all_tables_final(table_1, dict1, comrads_list = comrads_id_list, name_root = 'final3_test')
# выгружает в папку много файлов с именами студентов, that's it.

In [ ]:
# далее рисовалка прогресса заполнения.

In [5]:
table_1 = pd.read_excel(r'C:\Users\Professional\Downloads\final.xlsx') # я скачивал свою таблицу

In [6]:
def bar_comments(table_1):
    return (table_1.groupby('comrade').agg('count').iloc[:,1] / table_1.groupby('comrade').agg('count').iloc[:,0]).sort_values(ascending = False).plot.bar()

In [41]:
bar_comments(table_1) # эти функции рисуют таблицы успеваемости по некоторым колонкам

In [8]:
def bar_representation(table_test):
    return (table_test.drop('comrade', 1).notna().groupby(table_1.comrade).sum() / table_test.drop('comrade', 1).isna().groupby(table_1.comrade).count()).iloc[:,20].sort_values(ascending = False).plot.bar()

In [40]:
bar_representation(table_1)

In [76]:
# На вход - DataFrame. Он сохраняет в файлы excel с именем студента все отрицательные отзывы с рандомизацией для каждого человека.
def all_tables_final_negative(table, comrads_list = comrads_id_list, name_root = 'all_final'):
    try:
        os.mkdir(name_root)
    except:
        pass
    for i in comrads.iloc[comrads_id_list, 0].tolist():
        list_of_tables_final_one = []          
        list_of_tables_final_one.append(table[table.iloc[:, 1] == 'о'].sample(frac = 1))
        table_list_to_xlsx(list_of_tables_final_one, i, name_root)

In [77]:
all_tables_final_negative(table_1, name_root = 'negativeTest') #работает

In [ ]:
# писали люди, и я решил распределить оставшихся психологов. Не, ну а что...

In [749]:
razLess = all_ids_by_pages_unique(all_ids_by_pages(all_pages))

In [748]:
razMore = all_id_unique

In [753]:
ids_for_left_comrades = list(set(razMore) - set(razLess))

In [761]:
for i in ids_for_left_comrades:
    table_from_id_psycho(i)

In [765]:
def list_of_tables_left(all_ids):
    table = []
    for i in all_ids:
        i = i.strip('/')
        table.append(pd.read_excel(r'C:\Users\Professional\Jupyter Notebooks\parsPsy_left_comrades\{}.xlsx'.format(i)))
    return table

In [766]:
def table_list_to_xlsx_left(table, name):
    os.mkdir(str(name))
    pd.concat(table, axis = 0).to_excel(r'C:\Users\Professional\Jupyter Notebooks\{}\{}.xlsx'.format(name, name), index = False)

In [767]:
table_list_to_xlsx_left(list_of_tables_left(ids_for_left_comrades), '36')

In [769]:
table_left = pd.read_excel(r'C:\Users\Professional\Jupyter Notebooks\36\36.xlsx')

In [852]:
list_of_numbers = table_left.groupby('Link', sort = False).count()['Comments'].values

In [819]:
# я тогда приустал, но заставил себя написать какой-то сложный без необходимости генератор для распределения работы. Сейчас он не работает, потому что часть говнокода я убрал.

In [854]:
list_of_all_num = []
for i in range(1,7):
    list_of_all_num.extend([i] * list_of_sum[i - 1])

In [855]:
final_list_of_num = list_of_all_num

In [857]:
table_left['comrades'] = final_list_of_num

In [ ]:
table_left.to_excel(r'C:\Users\Professional\Jupyter Notebooks\final_left\final_left.xlsx', index = False)

In [ ]:
# ниже я чистил данные после полного заполнения.

In [38]:
clean = pd.read_excel(r'C:\Users\Professional\Downloads\final_left.xlsx', index = False)

In [39]:
clean.isna().sum()

In [79]:
# clean[clean.iloc[:, 0].isna()]
# clean[clean.iloc[:, 4].isna()]
# clean[clean.iloc[:, 5].isna()].index + 2
a = 'количество онлайн-консультаций (число)'
b = 'Количество отзывов'
c = 'количество обратившихся'
d = 'Количество публикаций на b17'
clean[clean['количество курсов повышения квалификации (число)'].isna()].index + 2

Int64Index([123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 228, 229,
            230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240],
           dtype='int64')

In [49]:
clean[clean.loc['Количество публикаций на b17':''].isna()]

,Unnamed: 0,"Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный)",ответ психолога на отзыв,ссылка на профиль\n,пол (м или ж),возраст,"опыт работы психологом (количество лет, число)",Город,Количество публикаций на b17,Количество отзывов,...,Было ли высшее образование до психологического (ДА/нет),стоимость одной очной консультации (в рублях),"Проблемы с которыми работает (текст, через точку с запятой)","Психологические школы (текст, через точку с запятой)",Полный текст самопрезентации психолога,comrade,Заполненность описания психологов,Заполненность маркировки отзывов,Описание психологов в основной таблице,Маркировка психологов в основной таблице
139,Обратился к Павлу в трудную минуту и не пожале...,п,Благодарю. Я высоко ценю Вашу оценку!,https://www.b17.ru/stegniy/,NaN,56.0,7,Кременчуг,184.0,12,...,ДА,200.0,"самооценка, внутренний конфликт; семейная псих...",Нейро-лингвистическое программирование; Психо...,"Практический психолог, психотерапевт, мастер с...",3,NaN,NaN,NaN,NaN
151,Спасибо Анастасии! Очень выручила меня! Всем с...,п,Сердцем- всегда на связи:),https://www.b17.ru/avetatur/,NaN,NaN,16,Сургут,72.0,5,...,нет,3000.0,напряжённые отношения и агрессия в паре; разли...,Арт-терапия;Гипнотерапия;Коучинг;Нейро-лингвис...,"Здравствуйте, меня зовут Анастасия.\nСамое важ...",3,NaN,NaN,NaN,NaN


In [45]:
clean[clean.iloc[:, 0].str.len() < 10]

,Unnamed: 0,"Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный)",ответ психолога на отзыв,ссылка на профиль\n,пол (м или ж),возраст,"опыт работы психологом (количество лет, число)",Город,Количество публикаций на b17,Количество отзывов,...,Было ли высшее образование до психологического (ДА/нет),стоимость одной очной консультации (в рублях),"Проблемы с которыми работает (текст, через точку с запятой)","Психологические школы (текст, через точку с запятой)",Полный текст самопрезентации психолога,comrade,Заполненность описания психологов,Заполненность маркировки отзывов,Описание психологов в основной таблице,Маркировка психологов в основной таблице
30,Спасибо,п,Удачи вам!,https://www.b17.ru/reuzkaja/,ж,41.0,20,Саратов,249.0,20,...,нет,1800.0,Преодоление неуверенности в себе; преодоление ...,Арт-терапия; гештальт-терапия; телесно-ориенти...,"В жизни каждого человека, независимо от пола, ...",1,NaN,NaN,NaN,NaN
33,Спасибо),п,Удачи и благополучия!,https://www.b17.ru/reuzkaja/,ж,41.0,20,Саратов,249.0,20,...,нет,1800.0,Преодоление неуверенности в себе; преодоление ...,Арт-терапия; гештальт-терапия; телесно-ориенти...,"В жизни каждого человека, независимо от пола, ...",1,NaN,NaN,NaN,NaN
304,Спасибо!,н,NaN,https://www.b17.ru/dobronevopros/,м,29.0,3,"Кривой Рог, Киев",18.0,12,...,нет,1095.0,Самопознание; саморазвитие; поиск себя; Неудов...,Арт-терапия; Когнитивно-поведенческая терапия ...,"Получить консультацию\n52 обращения\nПсихолог,...",6,NaN,NaN,NaN,NaN
